# 🎯 Day 3 프로젝트 솔루션: E2E MLOps Pipeline

## 📋 개요

이 노트북은 프로젝트의 완성된 솔루션입니다. 7개의 엔지니어링된 피처와 완전한 MLflow 통합을 포함합니다.

### ⚠️ 수정 사항 (2025-12-17)
- AWS S3 자격증명 파라미터 추가 (NoCredentialsError 해결)
- MLflow 모델 아티팩트 저장을 위한 AWS 환경변수 설정

---

## 1. 환경 설정

In [ ]:
!pip install kfp==2.7.0 -q

In [ ]:
import os
from kfp import dsl
from kfp.dsl import component, Input, Output, Dataset
from kfp import compiler

print(f"KFP SDK loaded")

In [ ]:
# ============================================================
# ⚠️ 설정 변경
# ============================================================

TEAM_NAME = "solution-team"

def get_current_namespace():
    namespace_path = "/var/run/secrets/kubernetes.io/serviceaccount/namespace"
    if os.path.exists(namespace_path):
        with open(namespace_path, "r") as f:
            return f.read().strip()
    return "kubeflow-user20"

USER_NAMESPACE = get_current_namespace()
MLFLOW_TRACKING_URI = f"http://mlflow-server.{USER_NAMESPACE}.svc.cluster.local:5000"

# ============================================================
# ⚠️ AWS 자격증명 설정
# ============================================================
AWS_ACCESS_KEY_ID = ""      # 본인의 Access Key
AWS_SECRET_ACCESS_KEY = ""  # 본인의 Secret Key
AWS_REGION = "ap-northeast-2"

print(f"Team: {TEAM_NAME}")
print(f"Namespace: {USER_NAMESPACE}")
print(f"MLflow: {MLFLOW_TRACKING_URI}")
print(f"AWS Key: {'✅' if AWS_ACCESS_KEY_ID else '❌ 미설정'}")

---

## 2. 컴포넌트 정의

### 2.1 Component 1: 데이터 로드

In [ ]:
@component(
    base_image="python:3.9-slim",
    packages_to_install=["pandas==2.0.3", "scikit-learn==1.3.2"]
)
def load_data(
    data_source: str,
    output_data: Output[Dataset]
):
    """California Housing 데이터셋 로드"""
    import pandas as pd
    from sklearn.datasets import fetch_california_housing
    
    print("=" * 60)
    print("  Step 1: Load Data")
    print("=" * 60)
    
    housing = fetch_california_housing(as_frame=True)
    df = housing.frame
    
    print(f"  Source: {data_source}")
    print(f"  Shape: {df.shape}")
    
    df.to_csv(output_data.path, index=False)
    print(f"  ✅ Data saved")

print("✅ load_data 정의 완료!")

### 2.2 Component 2: 전처리

In [ ]:
@component(
    base_image="python:3.9-slim",
    packages_to_install=["pandas==2.0.3", "scikit-learn==1.3.2", "numpy==1.24.3"]
)
def preprocess(
    input_data: Input[Dataset],
    X_train_out: Output[Dataset],
    X_test_out: Output[Dataset],
    y_train_out: Output[Dataset],
    y_test_out: Output[Dataset],
    test_size: float = 0.2
):
    """데이터 전처리"""
    import pandas as pd
    from sklearn.model_selection import train_test_split
    from sklearn.preprocessing import StandardScaler
    
    print("=" * 60)
    print("  Step 2: Preprocess")
    print("=" * 60)
    
    df = pd.read_csv(input_data.path)
    
    X = df.drop(columns=['MedHouseVal'])
    y = df['MedHouseVal']
    
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=test_size, random_state=42
    )
    
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    X_train_df = pd.DataFrame(X_train_scaled, columns=X_train.columns)
    X_test_df = pd.DataFrame(X_test_scaled, columns=X_test.columns)
    
    print(f"  Train: {X_train_df.shape}, Test: {X_test_df.shape}")
    
    X_train_df.to_csv(X_train_out.path, index=False)
    X_test_df.to_csv(X_test_out.path, index=False)
    pd.DataFrame(y_train).to_csv(y_train_out.path, index=False)
    pd.DataFrame(y_test).to_csv(y_test_out.path, index=False)
    
    print(f"  ✅ Preprocessing completed")

print("✅ preprocess 정의 완료!")

### 2.3 Component 3: 피처 엔지니어링 (7개 피처 - 완성 버전)

In [ ]:
@component(
    base_image="python:3.9-slim",
    packages_to_install=["pandas==2.0.3", "numpy==1.24.3"]
)
def feature_engineering(
    X_train_in: Input[Dataset],
    X_test_in: Input[Dataset],
    X_train_out: Output[Dataset],
    X_test_out: Output[Dataset]
) -> int:
    """피처 엔지니어링 (완성 버전 - 7개 피처)"""
    import pandas as pd
    import numpy as np
    
    print("=" * 60)
    print("  Step 3: Feature Engineering (Solution)")
    print("=" * 60)
    
    X_train = pd.read_csv(X_train_in.path)
    X_test = pd.read_csv(X_test_in.path)
    
    original_cols = list(X_train.columns)
    print(f"  Original features: {len(original_cols)}")
    
    def add_features(df):
        # 1. 침실 비율
        df['bedroom_ratio'] = df['AveBedrms'] / (df['AveRooms'] + 1e-6)
        
        # 2. 인당 방 수
        df['rooms_per_person'] = df['AveRooms'] / (df['AveOccup'] + 1e-6)
        
        # 3. 인구 밀도
        df['population_density'] = df['Population'] * df['AveOccup']
        
        # 4. 소득 대비 주택 연령
        df['income_age_ratio'] = df['MedInc'] / (df['HouseAge'] + 1)
        
        # 5. Bay Area 거리
        df['dist_to_bay'] = np.sqrt(
            (df['Latitude'] - 37.87)**2 + 
            (df['Longitude'] + 122.27)**2
        )
        
        # 6. LA 거리
        df['dist_to_la'] = np.sqrt(
            (df['Latitude'] - 34.05)**2 + 
            (df['Longitude'] + 118.24)**2
        )
        
        # 7. 해안 근접도 (경도 기준)
        df['coastal_proximity'] = np.abs(df['Longitude'] + 122)
        
        return df
    
    X_train_fe = add_features(X_train)
    X_test_fe = add_features(X_test)
    
    new_cols = [c for c in X_train_fe.columns if c not in original_cols]
    print(f"  New features ({len(new_cols)}): {new_cols}")
    print(f"  Total features: {len(X_train_fe.columns)}")
    
    X_train_fe.to_csv(X_train_out.path, index=False)
    X_test_fe.to_csv(X_test_out.path, index=False)
    
    print(f"  ✅ Feature engineering completed (7 new features)")
    
    return len(new_cols)

print("✅ feature_engineering 정의 완료! (7개 피처)")

### 2.4 Component 4: 모델 학습 (AWS 자격증명 포함)

In [ ]:
@component(
    base_image="python:3.9-slim",
    packages_to_install=[
        "pandas==2.0.3",
        "scikit-learn==1.3.2",
        "mlflow==2.9.2",
        "numpy==1.24.3",
        "boto3"
    ]
)
def train_model(
    X_train: Input[Dataset],
    X_test: Input[Dataset],
    y_train: Input[Dataset],
    y_test: Input[Dataset],
    mlflow_tracking_uri: str,
    experiment_name: str,
    team_name: str,
    # ✅ AWS 자격증명 파라미터
    aws_access_key_id: str,
    aws_secret_access_key: str,
    aws_region: str = "ap-northeast-2",
    n_estimators: int = 100,
    max_depth: int = 10
) -> str:
    """모델 학습 및 MLflow 기록 (완성 버전)"""
    import pandas as pd
    import numpy as np
    import mlflow
    from sklearn.ensemble import RandomForestRegressor
    from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
    import os
    
    print("=" * 60)
    print(f"  Step 4: Train Model - {team_name}")
    print("=" * 60)
    
    # ✅ AWS 자격증명 설정
    os.environ['AWS_ACCESS_KEY_ID'] = aws_access_key_id
    os.environ['AWS_SECRET_ACCESS_KEY'] = aws_secret_access_key
    os.environ['AWS_DEFAULT_REGION'] = aws_region
    print(f"  AWS Credentials: ✅ 설정됨")
    
    X_train_df = pd.read_csv(X_train.path)
    X_test_df = pd.read_csv(X_test.path)
    y_train_df = pd.read_csv(y_train.path)
    y_test_df = pd.read_csv(y_test.path)
    
    print(f"  Training data: {X_train_df.shape}")
    print(f"  Test data: {X_test_df.shape}")
    
    os.environ['MLFLOW_TRACKING_URI'] = mlflow_tracking_uri
    mlflow.set_tracking_uri(mlflow_tracking_uri)
    mlflow.set_experiment(experiment_name)
    
    with mlflow.start_run(run_name=f"{team_name}-run") as run:
        run_id = run.info.run_id
        print(f"  MLflow Run ID: {run_id}")
        
        mlflow.log_params({
            "n_estimators": n_estimators,
            "max_depth": max_depth,
            "random_state": 42,
            "n_features": X_train_df.shape[1],
            "n_samples_train": X_train_df.shape[0],
            "n_samples_test": X_test_df.shape[0]
        })
        mlflow.set_tag("team", team_name)
        mlflow.set_tag("pipeline", "solution")
        
        print(f"  Training RandomForest...")
        model = RandomForestRegressor(
            n_estimators=n_estimators,
            max_depth=max_depth,
            random_state=42,
            n_jobs=-1
        )
        model.fit(X_train_df, y_train_df.values.ravel())
        
        y_pred = model.predict(X_test_df)
        
        r2 = r2_score(y_test_df, y_pred)
        rmse = np.sqrt(mean_squared_error(y_test_df, y_pred))
        mae = mean_absolute_error(y_test_df, y_pred)
        
        mlflow.log_metrics({"r2": r2, "rmse": rmse, "mae": mae})
        
        print(f"  Performance:")
        print(f"    - R2 Score: {r2:.4f}")
        print(f"    - RMSE: {rmse:.4f}")
        print(f"    - MAE: {mae:.4f}")
        
        # 피처 중요도
        feature_importance = dict(zip(
            X_train_df.columns,
            model.feature_importances_
        ))
        sorted_importance = sorted(
            feature_importance.items(),
            key=lambda x: x[1],
            reverse=True
        )[:5]
        
        print(f"  Top 5 Feature Importance:")
        for feat, imp in sorted_importance:
            safe_name = feat.replace(" ", "_")[:15]
            mlflow.log_metric(f"fi_{safe_name}", imp)
            print(f"    - {feat}: {imp:.4f}")
        
        # ✅ 모델 저장 (AWS 자격증명으로 S3에 저장)
        mlflow.sklearn.log_model(model, "model")
        print(f"  ✅ Model saved to MLflow (S3)")
    
    return run_id

print("✅ train_model 정의 완료! (AWS 자격증명 포함)")

### 2.5 Component 5: 모델 평가

In [ ]:
@component(
    base_image="python:3.9-slim",
    packages_to_install=["mlflow==2.9.2", "boto3"]
)
def evaluate_model(
    run_id: str,
    mlflow_tracking_uri: str,
    r2_threshold: float = 0.75
) -> str:
    """모델 평가 및 배포 결정"""
    import mlflow
    import os
    
    print("=" * 60)
    print("  Step 5: Evaluate Model")
    print("=" * 60)
    
    os.environ['MLFLOW_TRACKING_URI'] = mlflow_tracking_uri
    mlflow.set_tracking_uri(mlflow_tracking_uri)
    
    run = mlflow.get_run(run_id)
    r2 = run.data.metrics.get("r2", 0)
    
    print(f"  R2: {r2:.4f}, Threshold: {r2_threshold}")
    
    if r2 >= r2_threshold:
        print(f"  ✅ DEPLOY")
        return "deploy"
    else:
        print(f"  ⚠️ SKIP")
        return "skip"

print("✅ evaluate_model 정의 완료!")

### 2.6 Component 6: 배포

In [ ]:
@component(base_image="python:3.9-slim")
def deploy_model(run_id: str, model_name: str, namespace: str):
    """모델 배포 정보 출력"""
    print("=" * 60)
    print("  Step 6: Deploy Model")
    print("=" * 60)
    print(f"  Model: {model_name}")
    print(f"  Namespace: {namespace}")
    print(f"  Endpoint: http://{model_name}.{namespace}.svc.cluster.local/v1/models/{model_name}:predict")
    print(f"  ✅ Deployment completed")

print("✅ deploy_model 정의 완료!")

### 2.7 Component 7: 알림

In [ ]:
@component(base_image="python:3.9-slim")
def send_alert(run_id: str, team_name: str):
    """성능 미달 알림"""
    print("=" * 60)
    print(f"  Alert - {team_name}")
    print("=" * 60)
    print(f"  ⚠️ Model did not meet performance threshold")
    print(f"  Recommendations:")
    print(f"    1. Add more features")
    print(f"    2. Tune hyperparameters")
    print(f"    3. Try different algorithms")

print("✅ send_alert 정의 완료!")

---

## 3. 파이프라인 정의

In [ ]:
@dsl.pipeline(
    name="Project Pipeline (Solution)",
    description="Solution: E2E ML Pipeline with 7 engineered features + AWS credentials"
)
def project_pipeline(
    data_source: str = "sklearn",
    team_name: str = "solution-team",
    experiment_name: str = "solution-experiment",
    model_name: str = "solution-model",
    namespace: str = USER_NAMESPACE,
    mlflow_tracking_uri: str = MLFLOW_TRACKING_URI,
    # ✅ AWS 자격증명 파라미터
    aws_access_key_id: str = "",
    aws_secret_access_key: str = "",
    aws_region: str = "ap-northeast-2",
    n_estimators: int = 100,
    max_depth: int = 10,
    r2_threshold: float = 0.75
):
    """프로젝트 솔루션 파이프라인"""
    
    load_task = load_data(data_source=data_source)
    
    preprocess_task = preprocess(
        input_data=load_task.outputs["output_data"]
    )
    
    feature_task = feature_engineering(
        X_train_in=preprocess_task.outputs["X_train_out"],
        X_test_in=preprocess_task.outputs["X_test_out"]
    )
    
    # ✅ AWS 자격증명 전달
    train_task = train_model(
        X_train=feature_task.outputs["X_train_out"],
        X_test=feature_task.outputs["X_test_out"],
        y_train=preprocess_task.outputs["y_train_out"],
        y_test=preprocess_task.outputs["y_test_out"],
        mlflow_tracking_uri=mlflow_tracking_uri,
        experiment_name=experiment_name,
        team_name=team_name,
        aws_access_key_id=aws_access_key_id,
        aws_secret_access_key=aws_secret_access_key,
        aws_region=aws_region,
        n_estimators=n_estimators,
        max_depth=max_depth
    )
    
    evaluate_task = evaluate_model(
        run_id=train_task.output,
        mlflow_tracking_uri=mlflow_tracking_uri,
        r2_threshold=r2_threshold
    )
    
    with dsl.If(evaluate_task.output == "deploy"):
        deploy_model(
            run_id=train_task.output,
            model_name=model_name,
            namespace=namespace
        )
    
    with dsl.If(evaluate_task.output == "skip"):
        send_alert(
            run_id=train_task.output,
            team_name=team_name
        )

print("✅ project_pipeline (Solution) 정의 완료!")

---

## 4. 파이프라인 컴파일

In [ ]:
pipeline_file = "solution_pipeline.yaml"

compiler.Compiler().compile(
    pipeline_func=project_pipeline,
    package_path=pipeline_file
)

print("=" * 60)
print("  Solution Pipeline 컴파일 완료!")
print("=" * 60)
print(f"  파일: {pipeline_file}")
print("")
print("  ⚠️ 파이프라인 실행 시 입력 필수:")
print(f"    - aws_access_key_id")
print(f"    - aws_secret_access_key")
print("")
print("  📋 솔루션 포함 사항:")
print("    - 7개 엔지니어링 피처")
print("    - 완전한 MLflow 통합")
print("    - 피처 중요도 로깅")
print("    - AWS S3 모델 저장")
print("=" * 60)

---

## 5. (선택) 직접 실행

In [ ]:
RUN_PIPELINE = False

if RUN_PIPELINE:
    from kfp.client import Client
    
    client = Client()
    
    run = client.create_run_from_pipeline_func(
        project_pipeline,
        arguments={
            "team_name": TEAM_NAME,
            "experiment_name": f"{TEAM_NAME}-experiment",
            "namespace": USER_NAMESPACE,
            "mlflow_tracking_uri": MLFLOW_TRACKING_URI,
            "aws_access_key_id": AWS_ACCESS_KEY_ID,
            "aws_secret_access_key": AWS_SECRET_ACCESS_KEY,
            "aws_region": AWS_REGION,
        },
        experiment_name=f"{TEAM_NAME}-experiments"
    )
    
    print(f"✅ Run 시작: {run.run_id}")
else:
    print("ℹ️  RUN_PIPELINE = True로 변경하여 직접 실행하거나,")
    print("   Kubeflow Dashboard에서 yaml 파일을 업로드하세요.")